## Sentimental Analysis

> By : Binay Gurung, Kashish Jagtiani, Umakant Datt Bhatt

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My\ Drive/COMP4641\ Project/

/content/drive/My Drive/COMP4641 Project


In [0]:
!pip install vaderSentiment

In [0]:
import numpy as np
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from ast import literal_eval

In [0]:
df = pd.read_csv('data/top5_final_checkin_review_updated_new.csv')

In [0]:
review_cols = [
  'reviews2013',
  'reviews2014',
  'reviews2015'
]

count = 0
for review_year in review_cols:
  for i,bus in enumerate(df[review_year]):
    list_review = literal_eval(bus)
    df[review_year][i] = list_review

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [0]:
reviews = df['reviews2013'][0]
len(reviews)
reviews[1]

'Had to call the police to get a refund for my food we waited 15 minutes the food did not come. I asked for a refund they reefused so I had to call the police when the police got there they gave me my refund I have never had to call the police on any kind of establishment before.NEVER AGAIN WILL I GO HERE!!'

In [0]:
analyzer = SentimentIntensityAnalyzer()
def polarity(review):
  vs = analyzer.polarity_scores(review)
  return vs

In [0]:
review_cols = [
  'reviews2013',
  'reviews2014',
  'reviews2015'
]

sentiment_col = {}

for i,review_year in enumerate(review_cols):
  col = []
  for bus in df[review_year]:
    sentiment_scores = []
    list_review = bus
    for review in list_review:
      sentiment_scores.append(polarity(review))
    col.append(sentiment_scores)
  sentiment_col[review_year+'_sentiment'] = col


In [0]:
sentiment_col.keys()

dict_keys(['reviews2013_sentiment', 'reviews2014_sentiment', 'reviews2015_sentiment'])

In [0]:
len(sentiment_col['reviews2013_sentiment'])

7982

In [0]:
for key in sentiment_col.keys():
  df[key] = sentiment_col[key]

In [0]:
df = df.drop(['reviews2013','reviews2014','reviews2015','user_fans','checkin_dates','user_stars'],axis=1)

In [0]:
df.head(3)

,Unnamed: 0,Unnamed: 0.1,business_id,name,stars,review_count,is_open,categories,processed_city,reviews2013_sentiment,reviews2014_sentiment,reviews2015_sentiment
0,0,0,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,3.0,18,1,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"[(0.0, 0.877, 0.123, 0.9019), (0.0, 1.0, 0.0, ...","[(0.045, 0.743, 0.212, 0.9863)]",[]
1,1,1,q2KtxnPa6rPSpAlMbg9l8g,Nothing Bundt Cakes,4.0,174,1,"Desserts, Food, Cupcakes, Bakeries",Scottsdale,"[(0.0, 0.71, 0.29, 0.9885), (0.025, 0.79, 0.18...","[(0.0, 0.797, 0.203, 0.9524), (0.0, 0.862, 0.1...","[(0.031, 0.923, 0.045, 0.2716), (0.009, 0.823,..."
2,2,2,44YFU284Z3KDEy25QyVoUw,Nee House Chinese Restaurant,3.5,269,1,"Chinese, Restaurants",Phoenix,"[(0.018, 0.848, 0.134, 0.9324), (0.01, 0.76, 0...","[(0.0, 0.694, 0.306, 0.9283), (0.145, 0.828, 0...","[(0.084, 0.687, 0.229, 0.902), (0.119, 0.786, ..."


In [0]:
%cd sentimental\ analysis

/content/drive/My Drive/COMP4641 Project/sentimental analysis


In [0]:
df.to_csv('preliminary_sentiment_score.csv',header=True)

## Binary Score Classification

In [0]:
newdf = df.copy()

In [0]:
newdf.head(3)

,Unnamed: 0,Unnamed: 0.1,business_id,name,stars,review_count,is_open,categories,processed_city,reviews2013_sentiment,reviews2014_sentiment,reviews2015_sentiment
0,0,0,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,3.0,18,1,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"[(0.0, 0.877, 0.123, 0.9019), (0.0, 1.0, 0.0, ...","[(0.045, 0.743, 0.212, 0.9863)]",[]
1,1,1,q2KtxnPa6rPSpAlMbg9l8g,Nothing Bundt Cakes,4.0,174,1,"Desserts, Food, Cupcakes, Bakeries",Scottsdale,"[(0.0, 0.71, 0.29, 0.9885), (0.025, 0.79, 0.18...","[(0.0, 0.797, 0.203, 0.9524), (0.0, 0.862, 0.1...","[(0.031, 0.923, 0.045, 0.2716), (0.009, 0.823,..."
2,2,2,44YFU284Z3KDEy25QyVoUw,Nee House Chinese Restaurant,3.5,269,1,"Chinese, Restaurants",Phoenix,"[(0.018, 0.848, 0.134, 0.9324), (0.01, 0.76, 0...","[(0.0, 0.694, 0.306, 0.9283), (0.145, 0.828, 0...","[(0.084, 0.687, 0.229, 0.902), (0.119, 0.786, ..."


In [0]:
review_sentiment_cols = [
  'reviews2013_sentiment',
  'reviews2014_sentiment',
  'reviews2015_sentiment'
]
bin_sentiment_col = {}
for review_col in review_sentiment_cols:
  cols = []
  for bus in df[review_col]:
    bus_sentiments = []
    for review in bus:
      bus_sentiments.append(eval_score_to_bin(review))

    cols.append(bus_sentiments)
  
  bin_sentiment_col[review_col+'_bin_score'] = cols
  


In [0]:
def eval_score_to_bin(scores):
  neg,neu,pos,compound = scores
  if(compound>0.05): # positive sentiment
    return 1
  elif compound<-0.05: # negative sentiment
    return -1
  else:     # neutral sentiment
    return 0
  

In [0]:
for col in review_sentiment_cols:
  newdf[col+'_bin_score'] = bin_sentiment_col[col+'_bin_score']

In [0]:
newdf = newdf.drop(['reviews2015_sentiment','reviews2014_sentiment','reviews2013_sentiment'],axis=1)

In [0]:
newdf.head(3)

,Unnamed: 0,Unnamed: 0.1,business_id,name,stars,review_count,is_open,categories,processed_city,reviews2013_sentiment_bin_score,reviews2014_sentiment_bin_score,reviews2015_sentiment_bin_score
0,0,0,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,3.0,18,1,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"[1, 0]",[1],[]
1,1,1,q2KtxnPa6rPSpAlMbg9l8g,Nothing Bundt Cakes,4.0,174,1,"Desserts, Food, Cupcakes, Bakeries",Scottsdale,"[1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,2,2,44YFU284Z3KDEy25QyVoUw,Nee House Chinese Restaurant,3.5,269,1,"Chinese, Restaurants",Phoenix,"[1, 1, -1, 1, 0, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1...","[1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","[1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 0, 1, 1, 1..."


In [0]:
%cd ..

/content/drive/My Drive/COMP4641 Project


In [0]:
%ls final/

rate_of_checkin_per_6_months.csv


In [0]:
rate_check_df = pd.read_csv('final/rate_of_checkin_per_6_months.csv')

In [0]:
for col in review_sentiment_cols:
  rate_check_df[col+'_bin_score'] = bin_sentiment_col[col+'_bin_score']

In [0]:
rate_check_df = rate_check_df.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'],axis=1)

In [0]:
rate_check_df.head(5)

,business_id,name,stars,review_count,is_open,categories,processed_city,user_stars,user_fans,checkin_dates,reviews2013,reviews2014,reviews2015,rate_mid_2013,rate_start_2014,rate_mid_2014,rate_start_2015,rate_mid_2015,reviews2013_sentiment_bin_score,reviews2014_sentiment_bin_score,reviews2015_sentiment_bin_score
0,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,3.0,18,1,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"[3.76, 2.55, 3.68, 3.17, 3.68, 3.68, 2.31, 3.6...","[0, 5, 29, 0, 2, 0, 0, 92, 26, 0, 3, 0, 17, 17...","['2013-01-01 08:38:32', '2013-01-07 01:26:01',...",['Taco Bell has really improved over the years...,"[""Roses are Red\nViolets are Blue,\nI'd be tru...",[],-5,-3,0,-1,-4,"[1, 0]",[1],[]
1,q2KtxnPa6rPSpAlMbg9l8g,Nothing Bundt Cakes,4.0,174,1,"Desserts, Food, Cupcakes, Bakeries",Scottsdale,"[3.82, 3.55, 4.13, 5.0, 3.63, 5.0, 4.11, 4.75,...","[4, 1, 18, 0, 0, 0, 35, 0, 1, 43, 0, 10, 2, 7,...","['2013-01-11 21:29:14', '2013-01-22 22:08:56',...",['Thank you Nothing Bundt Cakes for donating m...,"[""I called ten other bakers and this is the on...","['I placed an on-line order for an 8"" chocolat...",-5,7,-6,-2,6,"[1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,44YFU284Z3KDEy25QyVoUw,Nee House Chinese Restaurant,3.5,269,1,"Chinese, Restaurants",Phoenix,"[4.7, 4.25, 3.93, 4.07, 3.57, 3.33, 4.13, 3.5,...","[0, 0, 111, 10, 4, 5, 0, 0, 11, 5, 3, 1, 1, 1,...","['2013-01-13 01:30:16', '2013-01-17 01:16:37',...","[""We eat here on a regular basis. It's like t...",['My mom loves this restaurant ! Good food ! B...,"[""3.5 *s. Had west lake beef soup (pretty goo...",10,4,-6,-2,0,"[1, 1, -1, 1, 0, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1...","[1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","[1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 0, 1, 1, 1..."
3,ysfeDcLAYk_MVzfBmyiSTg,Vals Getaway DES Cafeteria,3.0,14,1,"Restaurants, Food, Cafes",Phoenix,"[3.25, 2.75, 2.63, 4.18, 4.15, 4.41, 3.82, 4.5...","[0, 2, 9, 0, 0, 2, 0, 0, 9, 9, 0, 5, 2, 0]","['2013-06-21 19:33:23', '2014-03-12 22:42:46',...",[],['The employees are nice and accommodating and...,[],-1,3,1,-3,-1,[],"[1, 1, 1, 1, 1]",[]
4,nsNONDHbV7Vudqh21uicqw,Salt Cellar,3.5,526,1,"Seafood, Restaurants",Scottsdale,"[1.5, 3.7, 1.0, 2.17, 3.76, 3.33, 4.0, 3.37, 4...","[0, 4, 0, 0, 1, 0, 1, 0, 3, 30, 0, 0, 2, 0, 0,...","['2013-01-06 00:15:01', '2013-01-09 01:25:09',...","[""As a lover of seafood I have been wanting to...",['Came back to this famous place after years o...,"['Poor service, had to sit there and ask my wi...",-5,-12,14,10,-16,"[1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1...","[1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1..."


In [0]:
rate_check_df.to_csv('final/rate_sentiment_final.csv',header=True)